## Checklist

### 진행예정
 1) 추가필요 열: 날짜 \
 2) 타 플랫폼과 합쳤을 때 형식 통일하도록 data type과 Nan 처리방법 등 정의 \
 3) 메리어트 계열사 제외처리 필요
 

### 완료
 1) page바꿔가면서 했을 때 상품들 연속성 CHK (limit=50에 따른 상품 누락 여부 확인) -> 이상없음. \
 2) page=1, 2 이상 구분할 때 [4], [1]이 아니라 '전체'를 가져오게 해야겠다. 시간 저녁되니까 숫자로 안된다. -> 해결 \
 3) today 기준시 기준으로, 한국보다 9시간 느림 -> 한국시로 맞추기 -> 해결
 4) 리뷰 추가 필요 -> 해결
 5) headers에서 app-version 나중에 업데이트 필요할 수도 있음 -> 업데이트 이전의 app-version 넣어도 이상없음 -> 해결
### 추후 체크 디버깅 예상
 1) X

In [1]:
import pandas as pd
import numpy as np

In [2]:
# 1. 프로젝트 생성

In [2]:
# !rm -rf dailyhotel
!scrapy startproject dailyhotel

New Scrapy project 'dailyhotel', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/python3/notebook/project/crawling/crawling-repo-2/in_progress/dailyhotel

You can start your first spider with:
    cd dailyhotel
    scrapy genspider example example.com


In [3]:
!tree dailyhotel

dailyhotel
├── DailyhotelKeys.py
├── __pycache__
│   └── DailyhotelKeys.cpython-36.pyc
├── dailyhotel
│   ├── __init__.py
│   ├── __pycache__
│   │   ├── __init__.cpython-36.pyc
│   │   ├── items.cpython-36.pyc
│   │   ├── mongodb.cpython-36.pyc
│   │   ├── pipelines.cpython-36.pyc
│   │   └── settings.cpython-36.pyc
│   ├── items.py
│   ├── middlewares.py
│   ├── mongodb.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       ├── __pycache__
│       │   ├── __init__.cpython-36.pyc
│       │   └── spider.cpython-36.pyc
│       └── spider.py
├── dailyhotel.csv
└── scrapy.cfg

5 directories, 19 files


In [28]:
# 2. xpath 찾기: headers 설정

### hotel_key 가져오는 모듈 생성

In [10]:
# hotel key 얻는 모듈 생성 -> dailyhotel.py
def get_hotel_keys():
    import scrapy, requests
    from scrapy.http import TextResponse
    import datetime
    
    
    KST = datetime.timezone(datetime.timedelta(hours=9))
    today = datetime.datetime.now(tz=KST) # 한국시간으로 설정
    start_date= today.strftime("%Y-%m-%d") # 시작일 디폴트 = 오늘
    end_date = (today + datetime.timedelta(days=1)).strftime("%Y-%m-%d") # 종료일 디폴트 = 내일
    staydays = (datetime.datetime.strptime(end_date,'%Y-%m-%d').date() - \
                datetime.datetime.strptime(start_date,'%Y-%m-%d').date()).days # 숙박일수 디폴트 = 1박start_date, end_date
    persons, staydays = 1, 1
    
    
    # 호텔리스트 가져올 parent_url 생성
    page = 1
    hotelidx = []
    while True:
        parent_url = f"https://www.dailyhotel.com/newdelhi/goodnight/api/v4/stays/sales?dateCheckIn={start_date}&regionIdx=30070&shortCutType=hotel_resort&persons={persons}&stays={staydays}&details=true&regionStayType=hotel_resort&stayType=hotel,resort&page={page}&limit=50"
        headers = {'app-version': '2.2.14'}
        req = requests.get(parent_url, headers=headers)
        response = req.json()
        list_all = response['data']['staySaleSections']

        if len(list_all) > 0:
            for each in list_all:
                if each['sectionViewType'] == 'NORMAL_섹션뷰타입':
                    lists = each['staySales']
                    print('page=', str(page))
                    page += 1
        else:
            print('All registered hotels have been searched.') # 이거는 확인용
            break


        # parent_url에서 각 호텔들의 Index number 가져오기
        for each in lists:
            hotelidx.append(each['hotelIdx'])
    
    return hotelidx

### 가져온 hotel_keys로 xpath찾기

In [22]:
# 일단 hotel_keys 받고 url 만들어서 yield시킬 예정
# 테스트
idx = hotelidx[0]
staydays = 1
start_date = "2021-03-10"


hotel_url = f'https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/{idx}?stays={staydays}&dateCheckIn={start_date}&regionStayType=all'
headers = {
    'app-version': '2.2.13'
}
req = requests.get(hotel_url, headers=headers)
response = TextResponse(req.url, body=req.text, encoding='utf-8')
resp_content = response.json()


#name
name = resp_content['data']['name'] 
#level
level = resp_content['data']['stayGradeName'] 
#score
try:
    score = round(resp_content['data']['rating']['values']/100*5, 1) 
except:
    score = '점수없음'
# location
location = resp_content['data']['address']
# roomtype
roomtype = []
for each in resp_content['data']['rooms']:
    roomtype.append(each['roomName'])
#price
price = []
for each in resp_content['data']['rooms']:
    price.append(each['amount']['discountAverage'])
#review
review = resp_content['data']['statistic']['reviewScoreTotalCount']
#link
hotel_desc_url = f"https://www.dailyhotel.com/stays/{idx}?dateCheckIn={start_date}&stays={staydays}" # 크롤링이랑 예약링크 다름 -> request url과 다르도록!

# print(name)
# print(level)
# print(score)
# print(location)
# print(roomtype)
# print(price)
# print(review)
# print(hotel_desc_url)

1491

In [12]:
# 3. item.py

In [3]:
%%writefile dailyhotel/dailyhotel/items.py
import scrapy


class DailyhotelItem(scrapy.Item):
    date = scrapy.Field()
    platform = scrapy.Field()
    name = scrapy.Field()
    level = scrapy.Field()
    score = scrapy.Field()
    review_count = scrapy.Field()
    location = scrapy.Field()
    room_type = scrapy.Field()
    price = scrapy.Field()
    link = scrapy.Field()

Overwriting dailyhotel/dailyhotel/items.py


# 4. spider.py

In [4]:
%%writefile dailyhotel/dailyhotel/spiders/spider.py
import scrapy, requests
import json
import datetime
from DailyhotelKeys import get_hotel_keys
from dailyhotel.items import DailyhotelItem


class DailyhotelSpider(scrapy.Spider):
    name = "Dailyhotel"
    allow_domain=["dailyhotel.com"]
    KST = datetime.timezone(datetime.timedelta(hours=9))
    today = datetime.datetime.now(tz=KST) # 한국시간으로 설정
    start_date= today.strftime("%Y-%m-%d") # 시작일 디폴트 = 오늘
    end_date = (today + datetime.timedelta(days=1)).strftime("%Y-%m-%d") # 종료일 디폴트 = 내일
    staydays = (datetime.datetime.strptime(end_date,'%Y-%m-%d').date() - \
                datetime.datetime.strptime(start_date,'%Y-%m-%d').date()).days # 숙박일수 디폴트 = 1박start_date, end_date
    persons, staydays = 2, 1
    start_urls= ["https://www.dailyhotel.com/"]
    
    
    def parse(self, response):
        hotel_idx = get_hotel_keys()
        KST = datetime.timezone(datetime.timedelta(hours=9))
        today = datetime.datetime.now(tz=KST) # 한국시간으로 설정
        start_date= today.strftime("%Y-%m-%d") # 시작일 디폴트 = 오늘
        links = []
        start_urls = "https://www.dailyhotel.com/"
        headers = {'app-version': '2.2.14',
                   'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.192 Safari/537.36',
                  }
        for idx in hotel_idx:
            hotel_url = start_urls + f'newdelhi/goodnight/api/v9/hotel/{idx}?stays=1&dateCheckIn={start_date}&regionStayType=all'
            links.append(hotel_url)   
        for link in links:
            yield scrapy.Request(link, headers=headers, callback=self.parse_content)
    
    
    def parse_content(self, response):
        KST = datetime.timezone(datetime.timedelta(hours=9))
        today = datetime.datetime.now(tz=KST) # 한국시간으로 설정
        start_date= today.strftime("%Y-%m-%d") # 시작일 디폴트 = 오늘  
        resp_content = response.json()
        staydays=1        
        #date
        date = start_date
        #name
        name = resp_content['data']['name'] 
        #level
        level = resp_content['data']['stayGradeName'] 
        #score
        try:
            score = float(round(resp_content['data']['rating']['values']/100*5, 1))
        except:
            score = float(0)
        # location
        location = resp_content['data']['address']
        # roomtype
        roomtypes = []
        for each in resp_content['data']['rooms']:
            roomtypes.append(each['roomName'])
        #price
        prices = []
        for each in resp_content['data']['rooms']:
            prices.append(int(each['amount']['discountAverage']))
        #review
        review = resp_content['data']['statistic']['reviewScoreTotalCount']
        #link
        hotel_id = resp_content['data']['idx']
        hotel_desc_url = f"https://www.dailyhotel.com/stays/{hotel_id}?dateCheckIn={start_date}&stays={staydays}" # 크롤링이랑 예약링크 다름 -> request url과 다르도록!
        #platform
        platform = "데일리호텔"

        
        item = DailyhotelItem()
        for i in range(len(roomtypes)):
            item['date'] = date
            item['platform'] = platform
            item['name'] = name
            item['level'] = level
            item['score'] = score
            item['review_count'] = review
            item['location'] = location
            item['room_type'] = roomtypes[i]
            item['price'] = prices[i]
            item['link'] = hotel_desc_url     
            yield item

Writing dailyhotel/dailyhotel/spiders/spider.py


In [30]:
# 5. settings.py: robots.txt 여부 파악

In [31]:
# 6. 프로젝트 실행

In [36]:
%pwd

'/home/ubuntu/python3/project/codes/crawling'

In [5]:
%%writefile run.sh
cd dailyhotel
rm dailyhotel.csv
scrapy crawl Dailyhotel -o dailyhotel.csv 

Overwriting run.sh


In [1]:
# 7. pipeline.py: 크롤링데이터 mongodb에 저장

In [2]:
# scrapy project에 pymongo module 생성

In [6]:
%%writefile dailyhotel/dailyhotel/mongodb.py
import pymongo


client = pymongo.MongoClient("mongodb://15.165.254.37:27017")
collection = client.hotel.hotel_info

Writing dailyhotel/dailyhotel/mongodb.py


In [26]:
# pipeline.py 설정

In [7]:
%%writefile dailyhotel/dailyhotel/pipelines.py
from itemadapter import ItemAdapter
from .mongodb import collection

class DailyhotelPipeline:
    def process_item(self, item, spider):
        data = {"date": item['date'], "platform": item['platform'], "name": item['name'],
                "level": item['level'], "score": item['score'], "review": item['review_count'],
                "location": item['location'], "room_type": item['room_type'],
                "price": item['price'], "link": item['link']}
        collection.insert(data)
        return item

Overwriting dailyhotel/dailyhotel/pipelines.py


In [9]:
# 8. settings.py -> pipeline 설정변경

```
ITEM_PIPELINES = {
   'dailyhotel.pipelines.DailyhotelPipeline': 300,
}
```

In [10]:
# 9. 프로젝트 실행!

In [ ]:
!/bin/bash run.sh

In [8]:
# 12초

In [9]:
import pandas as pd
data = pd.read_csv('dailyhotel/dailyhotel.csv', encoding='utf-8-sig')
print(data.shape)
data.tail()

(1807, 10)


,date,level,link,location,name,platform,price,review_count,room_type,score
1802,2021-03-12,특1급,https://www.dailyhotel.com/stays/27910?dateChe...,서울 용산구 청파로20길 95,그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산,데일리호텔,445000,745,[HOT DEAL!] 디럭스 스위트 2베드룸,4.8
1803,2021-03-12,특1급,https://www.dailyhotel.com/stays/27910?dateChe...,서울 용산구 청파로20길 95,그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산,데일리호텔,467500,745,디럭스 스위트 2베드룸,4.8
1804,2021-03-12,특1급,https://www.dailyhotel.com/stays/27910?dateChe...,서울 용산구 청파로20길 95,그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산,데일리호텔,506600,745,[HOT DEAL!ㅣ조식 2인 포함] 디럭스 스위트 2베드룸,4.8
1805,2021-03-12,특1급,https://www.dailyhotel.com/stays/27910?dateChe...,서울 용산구 청파로20길 95,그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산,데일리호텔,529100,745,[조식 PKG] 디럭스 스위트 2베드룸,4.8
1806,2021-03-12,특1급,https://www.dailyhotel.com/stays/27910?dateChe...,서울 용산구 청파로20길 95,그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산,데일리호텔,577000,745,[The Lounge l EFL 2인 포함PKG] 디럭스 스위트 2베드룸,4.8


In [ ]:
# 10. crontab 실행